# Fastq

In [53]:
def printb(s):
    print(str(s, 'utf-8'))

In [124]:
import subprocess
import re
from os import makedirs
from os.path import expanduser, join, basename
from glob import glob

base_dir = expanduser('~/PARKINSON/')
data_dir = 'data/ENP/2015_10_23_NGS1/UGB-CATG-AmpliconesB-01/'
bed_dir = join(base_dir, 'bed')
results_dir = join(base_dir, 'results/ENP/FECHA_DEL_ANALISIS/')

makedirs(results_dir, exist_ok=True)

fastqc_executable = expanduser('~/software/FastQC/fastqc')

fastq_filenames = [fn for fn in glob(data_dir + '*.fastq') if 'trimmed' not in fn]
fastq_filenames = [fn for fn in fastq_filenames if 'SAR099' in fn]  # Filtrado para probar con dos
fastq_filepaths = [join(base_dir, fn) for fn in fastq_filenames]
sample_ids = [re.search(r'(SAR\d+)-', s).group(1) for fp in fastq_filepaths]

In [84]:
def sample_results_dir(sample_id):
    return join(results_dir, sample_id)

for sample_id in set(sample_ids):
    makedirs(sample_results_dir(sample_id), exist_ok=True)

In [33]:
for fastq_filepath in fastq_filepaths:
    command = '{} {}'.format(fastqc_executable, fastq_filepath)
    out = subprocess.check_output(command.split(' '))
    print(out)

b'Analysis complete for SAR099-2015-01-00007_S8_L001_R2_001.fastq\n'
b'Analysis complete for SAR099-2015-01-00007_S8_L001_R1_001.fastq\n'


# Quality control y trimming de los fastq

In [89]:
fastq_mcf_executable = expanduser('~/software/ea-utils.1.1.2-537/fastq-mcf')
adapters_filepath = expanduser('~/PARKINSON/adaptadores_truseq/adaptadores.txt')

sample_id = 'SAR099'
trimmed_fastq_filenames = [join(sample_results_dir(sample_id),
                                basename(fastq_filepath.replace('.fastq', '.trimmed.fastq')))
                           for fastq_filepath in fastq_filepaths]

command_tmpl = '{} -o {} -o {} -l 50 -q 20 -x 10 -u -P 33 {} {} {}'
command = command_tmpl.format(fastq_mcf_executable, trimmed_fastq_filenames[0],
                              trimmed_fastq_filenames[1],
                              adapters_filepath, fastq_filepaths[0], fastq_filepaths[1])

out = subprocess.check_output(command.split(' '))
printb(out)

Scale used: 2.2
Filtering Illumina reads on purity field
Phred: 33
Threshold used: 140 out of 55722
Adapter Truseq_adapter_after_index (ATCTCGTATGCCGTCTTCTGCTTG): counted 23827 at the 'end' of '/home/juan/PARKINSON/data/ENP/2015_10_23_NGS1/UGB-CATG-AmpliconesB-01/SAR099-2015-01-00007_S8_L001_R1_001.fastq', clip set to 1
Adapter 1_rc (AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGTAGATCTCGGTGGTCGCCGTATCATT): counted 25804 at the 'end' of '/home/juan/PARKINSON/data/ENP/2015_10_23_NGS1/UGB-CATG-AmpliconesB-01/SAR099-2015-01-00007_S8_L001_R2_001.fastq', clip set to 1
Files: 2
Total reads: 55722
Too short after clip: 13618
Clipped 'end' reads (/home/juan/PARKINSON/data/ENP/2015_10_23_NGS1/UGB-CATG-AmpliconesB-01/SAR099-2015-01-00007_S8_L001_R2_001.fastq): Count 22664, Mean: 2.07, Sd: 3.94
Trimmed 24776 reads (/home/juan/PARKINSON/data/ENP/2015_10_23_NGS1/UGB-CATG-AmpliconesB-01/SAR099-2015-01-00007_S8_L001_R2_001.fastq) by an average of 5.83 bases on quality < 20
Clipped 'end' reads (/home/juan/PARKINSON

# Alignment to reference

Antes de usar el archivo de la referencia (fasta) crear un índice con
`bwa index -a bwtsw reference.fa`

In [134]:
sam_filepaths = []

for sample_id in set(sample_ids):
    print(sample_id)
    trimmed_fastqs = [fn for fn in trimmed_fastq_filenames if sample_id in fn]
    reference_genome_filename = expanduser('~/gatk_bundle/human_g1k_v37.fasta')
    sam_filepath = join(sample_results_dir(sample_id), '{}.sam').format(sample_id)
    sam_filepaths.append(sam_filepath)
    command = 'bwa mem -M {} {} {}'.format(reference_genome_filename,
                                           trimmed_fastqs[0], trimmed_fastqs[1])
    
    with open(sam_filepath, 'w+') as sam_file:
        print(command)
        subprocess.run(command.split(' '), stdout=sam_file)  # No redireccionó el STDOUT!

SAR099
bwa mem -M /home/juan/gatk_bundle/human_g1k_v37.fasta /home/juan/PARKINSON/results/ENP/FECHA_DEL_ANALISIS/SAR099/SAR099-2015-01-00007_S8_L001_R2_001.trimmed.fastq /home/juan/PARKINSON/results/ENP/FECHA_DEL_ANALISIS/SAR099/SAR099-2015-01-00007_S8_L001_R1_001.trimmed.fastq


Generamos 1 SAM de 2 fastqs del mismo chabón. Guardamos la info de la corrida para posterior uso en GATK (tal vez).

In [120]:
bam_filepaths = []
for sam_filepath in sam_filepaths:
    bam_filepath = sam_filepath.replace('.sam', '.bam')
    bam_filepaths.append(bam_filepath)
    library_id = 'ENPv1_LIB_00001'
    ngs_id = 'NGS_00001'

    picard_executable = expanduser('~/software/picard-tools-2.2.4/picard.jar ')
    read_groups_executable = (picard_executable + 'AddOrReplaceReadGroups')
    params = 'VALIDATION_STRINGENCY=LENIENT SO=coordinate CREATE_INDEX=TRUE'
    params += ' ID={ngs_id}.{library_id}.000000000-D0M8G.1.{sample_id} SM={sample_id} LB={library_id}'
    params += ' PL=ILLUMINA PU=000000000-D0M8G.1 I={sam_filepath} '
    params += ' O={bam_filepath}'

    params = params.format(**{
        'sample_id': sample_id,
        'library_id': library_id,
        'ngs_id': ngs_id,
        'sam_filepath': sam_filepath,
        'bam_filepath': bam_filepath,
    })

    command = 'java -jar {} {}'.format(read_groups_executable, params)
    print(command)
    subprocess.run(command.split(' '))

java -jar /home/juan/software/picard-tools-2.2.4/picard.jar AddOrReplaceReadGroups VALIDATION_STRINGENCY=LENIENT SO=coordinate CREATE_INDEX=TRUE ID=NGS_00001.ENPv1_LIB_00001.000000000-D0M8G.1.SAR099 SM=SAR099 LB=ENPv1_LIB_00001 PL=ILLUMINA PU=000000000-D0M8G.1 I=/home/juan/PARKINSON/results/ENP/FECHA_DEL_ANALISIS/SAR099/SAR099.sam  O=/home/juan/PARKINSON/results/ENP/FECHA_DEL_ANALISIS/SAR099/SAR099.bam


In [121]:
bam_filepaths

['/home/juan/PARKINSON/results/ENP/FECHA_DEL_ANALISIS/SAR099/SAR099.bam']

# Realign indels & Recalibrate BAM

Realineamiento en función de indels conocidos (usar archivos de indels en el bundle de gatk) ( esto está en el TP 9 del curso )



In [145]:
target_intervals_filenames = []
gatk_bundle_dir = expanduser('~/gatk_bundle')
gatk_executable = expanduser('~/software/GATK/GenomeAnalysisTK.jar')
realigner_executable = gatk_executable + ' -T RealignerTargetCreator'
thousand_genomes_indels = join(gatk_bundle_dir, '1000G_phase1.indels.b37.vcf')
mills_indels = join(gatk_bundle_dir, 'Mills_and_1000G_gold_standard.indels.b37.vcf')

for bam_filepath in bam_filepaths:
    target_intervals_filename = '{}.target_intervals.list'.format(join(sample_results_dir(sample_id), sample_id))
    target_intervals_filenames.append(target_intervals_filename)

    params = '-R {}'.format(reference_genome_filename)
    params += ' -I {}'.format(bam_filepath)
    for known_indels in [thousand_genomes_indels, mills_indels]:
        params += ' -known {}'.format(known_indels)
    params += ' -L {}'.format(join(bed_dir, 'disenio_ENP_illumina2_ordenado.bed'))
    params += ' -o {}'.format(target_intervals_filename)

    command = 'java -jar {} {}'.format(realigner_executable, params)
    print(command)
    # TODO: redirect stderr to a logfile. WARNING: not stdout, that's used by GATK
    subprocess.run(command.split(' '))

java -jar /home/juan/software/GATK/GenomeAnalysisTK.jar -T RealignerTargetCreator -R /home/juan/gatk_bundle/human_g1k_v37.fasta -I /home/juan/PARKINSON/results/ENP/FECHA_DEL_ANALISIS/SAR099/SAR099.bam -known /home/juan/gatk_bundle/1000G_phase1.indels.b37.vcf -known /home/juan/gatk_bundle/Mills_and_1000G_gold_standard.indels.b37.vcf -L /home/juan/PARKINSON/bed/disenio_ENP_illumina2_ordenado.bed -o /home/juan/PARKINSON/results/ENP/FECHA_DEL_ANALISIS/SAR099/SAR099.target_intervals.list


In [155]:
realigned_bam_filepaths = []

for target_intervals_filename in target_intervals_filenames:
    # bam_filepath define
    indel_realigner_executable = gatk_executable + ' -T IndelRealigner'
    realigned_bam_filepath = join(sample_results_dir(sample_id), '{}.realigned.bam'.format(sample_id))
    realigned_bam_filepaths.append(realigned_bam_filepath)
    params = '-R {}'.format(reference_genome_filename)
    params += ' -I {}'.format(bam_filepath)
    params += ' -targetIntervals {}'.format(target_intervals_filename)
    params += ' -o {}'.format(realigned_bam_filepath)
    
    command = 'java -jar {} {}'.format(indel_realigner_executable, params)
    print(command)
    # TODO: redirect stdout to a logfile
    subprocess.run(command.split(' '))

java -jar /home/juan/software/GATK/GenomeAnalysisTK.jar -T IndelRealigner -R /home/juan/gatk_bundle/human_g1k_v37.fasta -I /home/juan/PARKINSON/results/ENP/FECHA_DEL_ANALISIS/SAR099/SAR099.bam -targetIntervals /home/juan/PARKINSON/results/ENP/FECHA_DEL_ANALISIS/SAR099/SAR099.target_intervals.list -o /home/juan/PARKINSON/results/ENP/FECHA_DEL_ANALISIS/SAR099/SAR099.realigned.bam


In [150]:
realigned_bam_filepaths

['/home/juan/PARKINSON/results/ENP/FECHA_DEL_ANALISIS/SAR099/SAR099.realigned.bam']

In [158]:
recal_table_filepaths = []
dbSNP_filepath = join(gatk_bundle_dir, 'dbsnp_138.b37.vcf')
recal_executable = gatk_executable + ' -T BaseRecalibrator'

for realigned_bam_filepath in realigned_bam_filepaths:
    # sample_id define
    sample_dir = sample_results_dir(sample_id)
    recal_table_filepath = '{}.recalibration.table'.format(join(sample_dir, sample_id))
    recal_table_filepaths.append(recal_table_filepath)
    params = '-R {}'.format(reference_genome_filename)
    params += ' -I {}'.format(realigned_bam_filepath)
    for known_filename in [dbSNP_filepath, thousand_genomes_indels, mills_indels]:
        params += ' -knownSites {}'.format(known_filename)
    params += ' -L {}'.format(join(bed_dir, 'disenio_ENP_illumina2_ordenado.bed'))
    params += ' -o {}'.format(recal_table_filepath)

    command = 'java -jar {} {}'.format(recal_executable, params)
    print(command)
    # TODO: redirect stderr to logfile
    subprocess.run(command.split(' '))

java -jar /home/juan/software/GATK/GenomeAnalysisTK.jar -T BaseRecalibrator -R /home/juan/gatk_bundle/human_g1k_v37.fasta -I /home/juan/PARKINSON/results/ENP/FECHA_DEL_ANALISIS/SAR099/SAR099.realigned.bam -knownSites /home/juan/gatk_bundle/dbsnp_138.b37.vcf -knownSites /home/juan/gatk_bundle/1000G_phase1.indels.b37.vcf -knownSites /home/juan/gatk_bundle/Mills_and_1000G_gold_standard.indels.b37.vcf -L /home/juan/PARKINSON/bed/disenio_ENP_illumina2_ordenado.bed -o /home/juan/PARKINSON/results/ENP/FECHA_DEL_ANALISIS/SAR099/SAR099.recalibration.table


In [160]:
recal_table_filepaths
realigned_bam_filepaths

['/home/juan/PARKINSON/results/ENP/FECHA_DEL_ANALISIS/SAR099/SAR099.realigned.bam']

In [163]:
recalibrated_bam_filepaths = []
print_reads_executable = gatk_executable + ' -T PrintReads'

combo = zip(recal_table_filepaths, realigned_bam_filepaths)
for recal_table_filepath, realigned_bam_filepath in combo:
    # define sample_id or loop the sample_ids
    sample_dir = sample_results_dir(sample_id)
    recalibrated_bam_filepath = '{}.recalibrated.bam'.format(join(sample_dir, sample_id))
    recalibrated_bam_filepaths.append(recalibrated_bam_filepath)
    params = '-R {}'.format(reference_genome_filename)
    params += ' -I {}'.format(realigned_bam_filepath)
    params += ' -BQSR {}'.format(recal_table_filepath)
    params += ' -o {}'.format(recalibrated_bam_filepath)
    
    command = 'java -jar {} {}'.format(print_reads_executable, params)
    print(command)
    subprocess.run(command.split(' '))

java -jar /home/juan/software/GATK/GenomeAnalysisTK.jar -T PrintReads -R /home/juan/gatk_bundle/human_g1k_v37.fasta -I /home/juan/PARKINSON/results/ENP/FECHA_DEL_ANALISIS/SAR099/SAR099.realigned.bam -BQSR /home/juan/PARKINSON/results/ENP/FECHA_DEL_ANALISIS/SAR099/SAR099.recalibration.table -o /home/juan/PARKINSON/results/ENP/FECHA_DEL_ANALISIS/SAR099/SAR099.recalibrated.bam


# Variant calling

In [169]:
vcf_filepaths = []
haplo_caller_executable = gatk_executable + ' -T HaplotypeCaller'

for recalibrated_bam_filepath in recalibrated_bam_filepaths:
    vcf_filepath = '{}.raw.vcf'.format(join(sample_results_dir(sample_id), sample_id))
    vcf_filepaths.append(vcf_filepath)
    params = '-R {}'.format(reference_genome_filename)
    params += ' -L {}'.format(join(bed_dir, 'disenio_ENP_illumina2_ordenado.bed'))
    params += ' -I {}'.format(recalibrated_bam_filepath)
    params += ' --genotyping_mode DISCOVERY -stand_emit_conf 10 -stand_call_conf 30'
    params += ' -o {}'.format(vcf_filepath)
    
    command = 'java -jar {} {}'.format(haplo_caller_executable, params)
    print(command)
    subprocess.run(command.split(' '))    

java -jar /home/juan/software/GATK/GenomeAnalysisTK.jar -T HaplotypeCaller -R /home/juan/gatk_bundle/human_g1k_v37.fasta -L /home/juan/PARKINSON/bed/disenio_ENP_illumina2_ordenado.bed -I /home/juan/PARKINSON/results/ENP/FECHA_DEL_ANALISIS/SAR099/SAR099.recalibrated.bam --genotyping_mode DISCOVERY -stand_emit_conf 10 -stand_call_conf 30 -o /home/juan/PARKINSON/results/ENP/FECHA_DEL_ANALISIS/SAR099/SAR099.raw.vcf


In [170]:
gvcf_filepaths = []
haplo_caller_executable = gatk_executable + ' -T HaplotypeCaller'

for recalibrated_bam_filepath in recalibrated_bam_filepaths:
    gvcf_filepath = '{}.raw.gvcf'.format(join(sample_results_dir(sample_id), sample_id))
    gvcf_filepaths.append(vcf_filepath)
    params = '-R {}'.format(reference_genome_filename)
    params += ' -L {}'.format(join(bed_dir, 'disenio_ENP_illumina2_ordenado.bed'))
    params += ' -I {}'.format(recalibrated_bam_filepath)
    params += ' --genotyping_mode DISCOVERY -variant_index_parameter 128000'
    params += ' -stand_emit_conf 10 -stand_call_conf 30 -variant_index_type LINEAR'
    params += ' -o {}'.format(gvcf_filepath)
    command = 'java -jar {} {}'.format(haplo_caller_executable, params)
    print(command)
    subprocess.run(command.split(' '))

java -jar /home/juan/software/GATK/GenomeAnalysisTK.jar -T HaplotypeCaller -R /home/juan/gatk_bundle/human_g1k_v37.fasta -L /home/juan/PARKINSON/bed/disenio_ENP_illumina2_ordenado.bed -I /home/juan/PARKINSON/results/ENP/FECHA_DEL_ANALISIS/SAR099/SAR099.recalibrated.bam --genotyping_mode DISCOVERY -variant_index_parameter 128000 -stand_emit_conf 10 -stand_call_conf 30 -variant_index_type LINEAR -o /home/juan/PARKINSON/results/ENP/FECHA_DEL_ANALISIS/SAR099/SAR099.raw.gvcf
